In [1]:
import sys
try:
    import google.colab
    ENV_IS_CL = True
    !git clone --single-branch --branch network https://github.com/jameshalgren/wrf_hydro_nwm_public.git
    sys.path.append('/content/wrf_hydro_nwm_public/trunk/NDHMS/dynamic_channel_routing/src/python_framework')
    !pip install geopandas
    !pip install netcdf4
    #default recursion limit (~1000) is slightly too small for the deepest branches of the network
    sys.setrecursionlimit(6000) 
    #TODO: convert recursive functions to stack-based functions
except:
    ENV_IS_CL = False
    sys.path.append(r'../src/python_framework')


In [2]:
import networkbuilder as networkbuilder
import recursive_print
import os
import geopandas as gpd
import pandas as pd
import xarray as xr
# -*- coding: utf-8 -*-
"""NHD Network traversal

A demonstration version of this code is stored in this Colaboratory notebook:
https://colab.research.google.com/github/jameshalgren/wrf_hydro_nwm_public/blob/network/trunk/NDHMS/dynamic_channel_routing/notebooks/NHD_Network_Density_Analysis.ipynb#scrollTo=h_BEdl4LID34

"""
def do_network(
        geofile_path = None
        , title_string = None
        , layer_string = None
        , driver_string = None
        , key_col = None
        , downstream_col = None
        , length_col = None
        , terminal_code = None
        , verbose = False
        , debuglevel = 0
        ):

    # NOTE: these methods can lose the "connections" and "rows" arguments when
    # implemented as class methods where those arrays are members of the class.
    if verbose: print(title_string)
    if driver_string == 'NetCDF':
        geofile = xr.open_dataset(geofile_path)
        geofile_rows = (geofile.to_dataframe()).values
        # The xarray method for NetCDFs was implemented after the geopandas method for 
        # GIS source files. It's possible (probable?) that we are doing something 
        # inefficient by converting away from the Pandas dataframe.
        # TODO: Check the optimal use of the Pandas dataframe
        if debuglevel <= -1: print(f'reading -- dataset: {geofile_path}; layer: {layer_string}; driver: {driver_string}')
    else:
        if debuglevel <= -1: print(f'reading -- dataset: {geofile_path}; layer: {layer_string}; fiona driver: {driver_string}')
        geofile = gpd.read_file(geofile_path, driver=driver_string, layer=layer_string)
        geofile_rows = geofile.to_numpy()
    if debuglevel <= -2: geofile.plot() #TODO: WILL THIS WORK WITH NetCDF???
    if debuglevel <= -1: print(geofile.head()) #TODO: WILL THIS WORK WITH NetCDF???
    # Kick off recursive call for all connections and keys
    (connections) = networkbuilder.get_down_connections(
                    rows = geofile_rows
                    , key_col = key_col
                    , downstream_col = downstream_col
                    , length_col = length_col
                    , verbose = verbose
                    , debuglevel = debuglevel)
    
    (all_keys, ref_keys, headwater_keys
        , terminal_keys
        , terminal_ref_keys
        , circular_keys) = networkbuilder.determine_keys(
                    connections = connections
#                     , rows = geofile_rows
                    , key_col = key_col
                    , downstream_col = downstream_col
                    , terminal_code = terminal_code
                    , verbose = verbose
                    , debuglevel = debuglevel)
    
    (junction_keys) = networkbuilder.get_up_connections(
                    connections = connections
                    , terminal_code = terminal_code
                    , headwater_keys = headwater_keys
                    , terminal_keys = terminal_keys
                    , verbose = verbose
                    , debuglevel = debuglevel)
    return connections, all_keys, ref_keys, headwater_keys \
        , terminal_keys, terminal_ref_keys \
        , circular_keys, junction_keys

In [3]:
def do_print():    
    recursive_print.print_basic_network_info(
                    connections = connections_NHD
                    , headwater_keys = headwater_keys_NHD
                    , junction_keys = junction_keys_NHD
                    , terminal_keys = terminal_keys_NHD
                    , terminal_code = terminal_code_NHD
                    , verbose = True
                    )
    
    if 1 == 0: #THE RECURSIVE PRINT IS NOT A GOOD IDEA WITH LARGE NETWORKS!!!
        recursive_print.print_connections(
                    headwater_keys = headwater_keys_NHD
                    , down_connections = connections_NHD
                    , up_connections = connections_NHD
                    , terminal_code = terminal_code_NHD
                    , terminal_keys = terminal_keys_NHD
                    , terminal_ref_keys = terminal_ref_keys_NHD
                    , debuglevel = -2
                    )
    


### Two Real Networks

In [4]:
if ENV_IS_CL: root = '/content/wrf_hydro_nwm_public/trunk/NDHMS/dynamic_channel_routing/'
elif not ENV_IS_CL: root = os.path.dirname(os.path.abspath(''))
test_folder = os.path.join(root, r'test')
geo_input_folder = os.path.join(test_folder, r'input', r'geo', r'Channels')

"""##NHD Subset (Brazos/Lower Colorado)"""
Brazos_LowerColorado_ge5 = True
"""##NHD CONUS order 5 and greater"""
CONUS_ge5 = True
"""These are large -- be careful"""
CONUS_FULL_RES = True
CONUS_Named_Streams = False #create a subset of the full resolution by reading the GNIS field
CONUS_Named_combined = False #process the Named streams through the Full-Res paths to join the many hanging reaches

debuglevel = -1
verbose = True

# The CONUS_ge5 and Brazos_LowerColorado_ge5 datasets are included
# in the github test folder and are extracts from the NHD version 1.2 datasets
# from https://www.nohrsc.noaa.gov/pub/staff/keicher/NWM_live/web/data_tools/
#  
# The CONUS_FULL_RES file was generated from the RouteLink file in the parameter
# archive and converted to a compressed NetCDF via the following command:
# nccopy -d1 -s RouteLink_NWMv2.0_20190517_cheyenne_pull.nc RouteLink_NWMv2.0_20190517_cheyenne_pull.nc4s
# TODO: Explain CONUS_Named_Streams
# CONUS_Named_Streams was generated by intersecting the FULL_RES file ...
# of the data in the nohrsc-hosted archive but are too large to efficiently 
# package inside of the repository. 

if Brazos_LowerColorado_ge5:
    nhd_conus_file_path = os.path.join(geo_input_folder
            , r'NHD_BrazosLowerColorado_Channels.shp')
    key_col_NHD = 2
    downstream_col_NHD = 7
    length_col_NHD = 6
    terminal_code_NHD = 0
    title_string = 'Brazos + Lower Colorado\nNHD stream orders 5 and greater\n'
    title_string = 'CONUS Order 5 and Greater '
    driver_string = 'ESRI Shapefile'
    layer_string = 0

    Brazos_LowerColorado_ge5_values = do_network (nhd_conus_file_path
                , title_string = title_string
                , layer_string = layer_string
                , driver_string = driver_string
                , key_col = key_col_NHD
                , downstream_col = downstream_col_NHD
                , length_col = length_col_NHD
                , terminal_code = terminal_code_NHD
                , verbose = verbose
                , debuglevel = debuglevel)

if CONUS_ge5:
    nhd_conus_file_path = os.path.join(geo_input_folder
            , r'NHD_Conus_Channels.shp')
    key_col_NHD = 1
    downstream_col_NHD = 6
    length_col_NHD = 5
    terminal_code_NHD = 0
    title_string = 'CONUS Order 5 and Greater '
    driver_string = 'ESRI Shapefile'
    layer_string = 0

    CONUS_ge5_values = do_network (nhd_conus_file_path
                , title_string = title_string
                , layer_string = layer_string
                , driver_string = driver_string
                , key_col = key_col_NHD
                , downstream_col = downstream_col_NHD
                , length_col = length_col_NHD
                , terminal_code = terminal_code_NHD
                , verbose = verbose
                , debuglevel = debuglevel)

if CONUS_FULL_RES:
    # nhd_conus_file_path = '../../../../../../GISTemp/nwm_v12.gdb'
    nhd_conus_file_path = os.path.join(geo_input_folder
            , r'RouteLink_NWMv2.0_20190517_cheyenne_pull.nc')
    key_col_NHD = 0
    downstream_col_NHD = 2
    length_col_NHD = 10
    terminal_code_NHD = 0
    title_string = 'CONUS Full Resolution NWM v2.0'
    # driver_string = 'FileGDB'
    driver_string = 'NetCDF'
    # layer_string = 'channels_nwm_v12_routeLink'
    layer_string = 0

    CONUS_FULL_RES_values = do_network (nhd_conus_file_path
                , title_string = title_string
                , layer_string = layer_string
                , driver_string = driver_string
                , key_col = key_col_NHD
                , downstream_col = downstream_col_NHD
                , length_col = length_col_NHD
                , terminal_code = terminal_code_NHD
                , verbose = verbose
                , debuglevel = debuglevel)

CONUS Order 5 and Greater 
reading -- dataset: /home/jacob.hreha/nwm/trunk/NDHMS/dynamic_channel_routing/test/input/geo/Channels/NHD_BrazosLowerColorado_Channels.shp; layer: 0; fiona driver: ESRI Shapefile
   OBJECTID_1  OBJECTID  featureID  linkDim     link  order_  Length       to  \
0        1499     25442    3764288   460086  3764288       6  2150.0  3764296   
1        1500     25443    3764296   460088  3764296       6  1277.0  3765756   
2        1501     25444    3766380   460092  3766380       6   431.0  3766382   
3        1502     25445    3765756   460090  3765756       6  1274.0  3766380   
4        1503     25447    3765796   460180  3765796       6   219.0  3765798   

     MusK  MusX  ...  gages  NHDWaterbo      lat      lon   alt Kchan  \
0  3600.0   0.2  ...   None       -9999  29.0176 -96.0119  9.59     0   
1  3600.0   0.2  ...   None       -9999  29.0059 -96.0029  9.59     0   
2  3600.0   0.2  ...   None       -9999  28.9876 -95.9992  9.59     0   
3  3600.0   0.2

In [5]:
# def main():
if 1 == 1:
    """##TEST"""
    print("")
    print ('Executing Test')
    # Test data
    test_rows = [
    [0,456,None,0],
    [1,678,4,0],
    [2,394,0,0],
    [3,815,2,0],
    [4,798,0,0],
    [5,679,4,0],
    [6,394,0,0],
    [7,815,2,0],
    [8,841,None,0],
    [9,524,12,0],
    [10,458,9,0],
    [11,548,8,0],
    [12,543,8,0],
    [13,458,14,0],
    [14,548,10,0],
    [15,543,14,0],
]

    test_key_col = 0
    test_downstream_col = 2
    test_length_col = 1
    test_terminal_code = -999
    debuglevel = -2
    verbose = True

    (test_connections) = networkbuilder.get_down_connections(
                rows = test_rows
                , key_col = test_key_col
                , downstream_col = test_downstream_col
                , length_col = test_length_col
                , verbose = True
                , debuglevel = debuglevel
                )

    (test_all_keys, test_ref_keys, test_headwater_keys
     , test_terminal_keys
     , test_terminal_ref_keys
     , test_circular_keys) = networkbuilder.determine_keys(
                connections = test_connections
                
                , key_col = test_key_col
                , downstream_col = test_downstream_col
                , terminal_code = test_terminal_code
                , verbose = True
                , debuglevel = debuglevel
                )

    test_junction_keys = networkbuilder.get_up_connections(
                connections = test_connections
                , terminal_code = test_terminal_code
                , headwater_keys = test_headwater_keys
                , terminal_keys = test_terminal_keys
                , verbose = True
                , debuglevel = debuglevel
                )

    recursive_print.print_connections(
                headwater_keys = test_headwater_keys
                , down_connections = test_connections
                , up_connections = test_connections
                , terminal_code = test_terminal_code
                , terminal_keys = test_terminal_keys
                , terminal_ref_keys = test_terminal_ref_keys
                , debuglevel = debuglevel
                )

    recursive_print.print_basic_network_info(
                connections = test_connections
                , headwater_keys = test_headwater_keys
                , junction_keys = test_junction_keys
                , terminal_keys = test_terminal_keys
                , terminal_code = test_terminal_code
                , verbose = True
                , debuglevel = debuglevel
                )


# if __name__ == "__main__":
#     main()



Executing Test
down connections ...
found 16 segments
{0: {'downstream': None, 'length': 456}, 1: {'downstream': 4, 'length': 678}, 2: {'downstream': 0, 'length': 394}, 3: {'downstream': 2, 'length': 815}, 4: {'downstream': 0, 'length': 798}, 5: {'downstream': 4, 'length': 679}, 6: {'downstream': 0, 'length': 394}, 7: {'downstream': 2, 'length': 815}, 8: {'downstream': None, 'length': 841}, 9: {'downstream': 12, 'length': 524}, 10: {'downstream': 9, 'length': 458}, 11: {'downstream': 8, 'length': 548}, 12: {'downstream': 8, 'length': 543}, 13: {'downstream': 14, 'length': 458}, 14: {'downstream': 10, 'length': 548}, 15: {'downstream': 14, 'length': 543}}
down_connections complete
ref_keys ...
found 9 ref_keys
{0, 2, 4, None, 8, 9, 10, 12, 14}
ref_keys complete
{0, 2, 4, None, 8, 9, 10, 12, 14}
headwater_keys ...
found 8 headwater segments
{1, 3, 5, 6, 7, 11, 13, 15}
headwater_keys complete
terminal_keys ...
Non-standard terminal key None found in segment 0
Non-standard terminal key No

In [6]:
#Creates connections for the first time
import time

#list of the different key sets
#
# terminal_keys = Brazos_LowerColorado_ge5_values[4] 
# circular_keys = Brazos_LowerColorado_ge5_values[6]
# terminal_keys_super = terminal_keys - circular_keys
# con = Brazos_LowerColorado_ge5_values[0]
# terminal_code = terminal_code_NHD
# #
# terminal_keys = CONUS_ge5_values[4] 
# circular_keys = CONUS_ge5_values[6]
# terminal_keys_super = terminal_keys - circular_keys
# con = CONUS_ge5_values[0]
# #
terminal_keys_super = test_terminal_keys - test_circular_keys
con = test_connections
terminal_code = test_terminal_code

# terminal_keys = CONUS_FULL_RES_values[4] 
# circular_keys = CONUS_FULL_RES_values[6]
# terminal_keys_super = terminal_keys - circular_keys
# con = CONUS_FULL_RES_values[0]
# terminal_code = terminal_code_NHD
#orders of everything for computation in dictionary format
order_dict={}
junc_dict={}
head_dict={}
terminal_avg = {}

def recursive_junction_read (keys, iterator, con, network, terminal_code, verbose = False, debuglevel = 0):
    # print(keys)
    for key in keys:        
        result = 'node' + str(key) 
        order_dict[result] = iterator, nid
        ckey = key
        ukeys = con[key]['upstreams']
        # terminal key and node_order assignment 
        con[key].update({'nid': nid})
        con[key].update({'node_order' : iterator})
        n = []
        while not len(ukeys) >= 2 and not (ukeys == {terminal_code}):
            # the terminal code will indicate a headwater
            if debuglevel <= -2: print(ukeys)
            (ckey,) = ukeys
            ukeys = con[ckey]['upstreams']
            network['segment_count'] += 1
            #adds ordering for all nodes
            result = 'node' + str(ckey)
            order_dict[result] = iterator, nid
            #assignment of terminal key and node order adjustment for continuous serial orders without junctions
            con[ckey].update({'nid': nid})
            con[ckey].update({'node_order' : iterator+1+sum(n)})
            n.append(1)
            
        if len(ukeys) >= 2:
            if debuglevel <= -1: print(f"junction found at {ckey} with upstreams {ukeys}")
            network['segment_count'] += 1

            network['junction_count'] += 1 #the Terminal Segment
            #iterator adds 1 each iteration to provide a new order of computation for each junction section not each node or group of segments
            result_junc = 'junc' + str(key)
            junc_dict[result_junc] = iterator
            con[ckey].update({'nid': nid})
            
            recursive_junction_read (ukeys, iterator+1+sum(n), con, network, terminal_code, verbose, debuglevel)
            n.clear()
        elif ukeys == {terminal_code}:
            # print(f"headwater found at {ckey}")
            network['segment_count'] += 1
            #below adds headwaters to the headwater list
            result_head = 'head' + str(key)
            head_dict[result_head] = iterator
            con[ckey].update({'nid': nid})
            
      
def super_network_trace(nid, iterator, con, network, terminal_code, debuglevel = 0):
    # print(f'\ntraversing upstream on network {nid}:')
    try:
        network.update({'junction_count': 0})
        network.update({'segment_count': 0}) #the Terminal Segment
        
        recursive_junction_read([nid], iterator , con, network, terminal_code, debuglevel = debuglevel)
        
    except Exception as exc:
        print(exc)

super_networks = {terminal_key:{}
                        for terminal_key in terminal_keys_super}
debuglevel = 0

start_time = time.time()
for nid, network in super_networks.items():
    super_network_trace(nid, 0, con, network, terminal_code, debuglevel = debuglevel)
    

print("--- %s seconds ---" % (time.time() - start_time))



--- 0.00010943412780761719 seconds ---


# New Lengths Working
- Adjustsments can be made to reaches too small first by adding lengths to upstream or downstream nodes 
- Adjustsments are then made to reaches too large by slicing them into the number of reaches that satisfies max length
- New reaches are created and con values are updated to reflect this
- New reaches are given max+1 ID values to avoid any duplicate values

In [7]:
import math
new_con = con.copy()
def grow(p):
    for x , y in p.items():
        #specify min length of reaches
        if y['length'] < 400:
          
# for upstream junction with 1 downstream
# updates up and downstream connections
            if len(list(y['upstreams'])) > 1 and len(([y['downstream']])) == 1:
                for w , q in p.items():
                    if y['downstream'] == w:
                        if x in (q['upstreams'].union(y['upstreams'])):
                            i = (q['upstreams'].union(y['upstreams']))
                            i.remove(x)
                        else:
                            i = (q['upstreams'].union(y['upstreams']))
                        new_con.update({ w : {'downstream' : q['downstream'], 'length' : (q['length']+y['length']), 'upstreams' : i, 'nid': q['nid']}})
                    for e in list(i): 
                        if w == e:
                            new_con.update({ w : {'downstream' : y['downstream'], 'length' : q['length'], 'upstreams' : q['upstreams'], 'nid': q['nid']}})
                         #downstream
                del new_con[x]       

#for 1 upstream with 1 downstream and upstream is not a headwater 
            if len(([y['downstream']])) == 1 and len(list(y['upstreams'])) == 1 and (y['upstreams']) != {-999}:
                for d in list(y['upstreams']):
                    for f , g in p.items():
                        if d == f:
                            if x in list(g['upstreams']):
                                o = (g['upstreams'])
                                o.remove(x)
                            else:
                                o = (g['upstreams'])
                            new_con.update({ d : {'downstream' : y['downstream'], 'length' : ((g['length'])+y['length']), 'upstreams' : o, 'nid': g['nid']}})         
                del new_con[x]
       
grow(con)
for x,y in new_con.items():
    print(x,y)

0 {'downstream': None, 'length': 850, 'upstreams': {3, 4, 6, 7}, 'nid': 0}
1 {'downstream': 4, 'length': 678, 'upstreams': {-999}, 'nid': 0, 'node_order': 2}
3 {'downstream': 0, 'length': 815, 'upstreams': {-999}, 'nid': 0}
4 {'downstream': 0, 'length': 798, 'upstreams': {1, 5}, 'nid': 0}
5 {'downstream': 4, 'length': 679, 'upstreams': {-999}, 'nid': 0, 'node_order': 2}
6 {'downstream': 0, 'length': 394, 'upstreams': {-999}, 'nid': 0}
7 {'downstream': 0, 'length': 815, 'upstreams': {-999}, 'nid': 0}
8 {'downstream': None, 'length': 841, 'upstreams': {11, 12}, 'nid': 8, 'node_order': 0}
9 {'downstream': 12, 'length': 524, 'upstreams': {10}, 'nid': 8, 'node_order': 2}
10 {'downstream': 9, 'length': 458, 'upstreams': {14}, 'nid': 8, 'node_order': 3}
11 {'downstream': 8, 'length': 548, 'upstreams': {-999}, 'nid': 8, 'node_order': 1}
12 {'downstream': 8, 'length': 543, 'upstreams': {9}, 'nid': 8, 'node_order': 1}
13 {'downstream': 14, 'length': 458, 'upstreams': {-999}, 'nid': 8, 'node_orde

In [8]:
#This creates new reaches by dividing up reaches that are too large
import math
con = new_con.copy()
def shrink(p):
    for x , y in p.items():
        #Specify max length
        if y['length'] > 800:
            div = math.ceil(y['length']/800)
            count = [int(div)]
            n = div-1
            temp = {(max(con.keys())+1)}
            if sum(count) > 0:
                con.update({x : {'downstream' : y['downstream'], 'length' : int(y['length']/div), 'upstreams' : temp, 'nid': y['nid']}}) #downstream       
                for f in range(1,div):
                    if f == 1 and f != div-1:
                        con.update({(max(con.keys())+f) :{'downstream' : x, 'length' : int(y['length']/div), 'upstreams' :  {(max(con.keys())+2)}, 'nid': y['nid'] }}) # first node if only new node
                    if  f == div-1:
                        con.update({(max(con.keys())+f) :{'downstream' : x, 'length' : int(y['length']/div), 'upstreams' : y['upstreams'], 'nid': y['nid'] }}) #last node 
                     #new nodes first
                    if f != 1 and f != div-1:
                        con.update({(max(con.keys())+1) :{'downstream' : (max(con.keys())), 'length' : int(y['length']/div), 'upstreams' :{(max(con.keys())+2)}, 'nid': y['nid'] }}) #middle nodes
                count.append(-1)
            else:
                print("done")   
shrink(new_con)
for x,y in con.items():
    print(x,y)


0 {'downstream': None, 'length': 425, 'upstreams': {16}, 'nid': 0}
1 {'downstream': 4, 'length': 678, 'upstreams': {-999}, 'nid': 0, 'node_order': 2}
3 {'downstream': 0, 'length': 407, 'upstreams': {17}, 'nid': 0}
4 {'downstream': 0, 'length': 798, 'upstreams': {1, 5}, 'nid': 0}
5 {'downstream': 4, 'length': 679, 'upstreams': {-999}, 'nid': 0, 'node_order': 2}
6 {'downstream': 0, 'length': 394, 'upstreams': {-999}, 'nid': 0}
7 {'downstream': 0, 'length': 407, 'upstreams': {18}, 'nid': 0}
8 {'downstream': None, 'length': 420, 'upstreams': {19}, 'nid': 8}
9 {'downstream': 12, 'length': 524, 'upstreams': {10}, 'nid': 8, 'node_order': 2}
10 {'downstream': 9, 'length': 458, 'upstreams': {14}, 'nid': 8, 'node_order': 3}
11 {'downstream': 8, 'length': 548, 'upstreams': {-999}, 'nid': 8, 'node_order': 1}
12 {'downstream': 8, 'length': 543, 'upstreams': {9}, 'nid': 8, 'node_order': 1}
13 {'downstream': 14, 'length': 458, 'upstreams': {-999}, 'nid': 8, 'node_order': 5}
14 {'downstream': 10, 'len

# Recursive network builder with ordering
**Recursive functions capable of constructing the network from their given terminal, upstream, and downstream keys. **
*   Segments between nodes are tallied.
*   Junctions are outputted in junc_dict.
*   Each node is assigned a computational order for parallel processing assigned to node_order.
*   Inputted network keys (CONUS,BRAZOS,TEST) can be controlled under imports. 
*   IDs are passed to these functions from super_networks.items() to step through from the initial river outlets to the headwaters while labeling each order. 


---





In [11]:
#original 
import time

#list of the different key sets
#
# terminal_keys = Brazos_LowerColorado_ge5_values[4] 
# circular_keys = Brazos_LowerColorado_ge5_values[6]
# terminal_keys_super = terminal_keys - circular_keys
# con = Brazos_LowerColorado_ge5_values[0]
# terminal_code = terminal_code_NHD
# #
# terminal_keys = CONUS_ge5_values[4] 
# circular_keys = CONUS_ge5_values[6]
# terminal_keys_super = terminal_keys - circular_keys
# con = CONUS_ge5_values[0]
# #
# terminal_keys_super = test_terminal_keys - test_circular_keys
# # con = test_connections
# terminal_code = test_terminal_code

# terminal_keys = CONUS_FULL_RES_values[4] 
# circular_keys = CONUS_FULL_RES_values[6]
# terminal_keys_super = terminal_keys - circular_keys
# con = CONUS_FULL_RES_values[0]
# terminal_code = terminal_code_NHD
#orders of everything for computation in dictionary format
order_dict={}
junc_dict={}
head_dict={}
terminal_avg = {}

def recursive_junction_read (keys, iterator, con, network, terminal_code, verbose = False, debuglevel = 0):
    # print(keys)
    for key in keys:        
        result = 'node' + str(key) 
        order_dict[result] = iterator, nid
        ckey = key
        ukeys = con[key]['upstreams']
        # terminal key and node_order assignment 
        con[key].update({'nid': nid})
        con[key].update({'node_order' : iterator})
        n = []
        while not len(ukeys) >= 2 and not (ukeys == {terminal_code}):
            # the terminal code will indicate a headwater
            if debuglevel <= -2: print(ukeys)
            (ckey,) = ukeys
            ukeys = con[ckey]['upstreams']
            network['segment_count'] += 1
            #adds ordering for all nodes
            result = 'node' + str(ckey)
            order_dict[result] = iterator, nid
            #assignment of terminal key and node order adjustment for continuous serial orders without junctions
            con[ckey].update({'nid': nid})
            con[ckey].update({'node_order' : iterator+1+sum(n)})
            n.append(1)
            
        if len(ukeys) >= 2:
            if debuglevel <= -1: print(f"junction found at {ckey} with upstreams {ukeys}")
            network['segment_count'] += 1

            network['junction_count'] += 1 #the Terminal Segment
            #iterator adds 1 each iteration to provide a new order of computation for each junction section not each node or group of segments
            result_junc = 'junc' + str(key)
            junc_dict[result_junc] = iterator
            con[ckey].update({'nid': nid})
            
            recursive_junction_read (ukeys, iterator+1+sum(n), con, network, terminal_code, verbose, debuglevel)
            n.clear()
        elif ukeys == {terminal_code}:
            # print(f"headwater found at {ckey}")
            network['segment_count'] += 1
            #below adds headwaters to the headwater list
            result_head = 'head' + str(key)
            head_dict[result_head] = iterator
            con[ckey].update({'nid': nid})
            
      
def super_network_trace(nid, iterator, con, network, terminal_code, debuglevel = 0):
    # print(f'\ntraversing upstream on network {nid}:')
    try:
        network.update({'junction_count': 0})
        network.update({'segment_count': 0}) #the Terminal Segment
        
        recursive_junction_read([nid], iterator , con, network, terminal_code, debuglevel = debuglevel)
        
    except Exception as exc:
        print(exc)

super_networks = {terminal_key:{}
                        for terminal_key in terminal_keys_super}
debuglevel = 0

start_time = time.time()
for nid, network in super_networks.items():
    super_network_trace(nid, 0, con, network, terminal_code, debuglevel = debuglevel)
    

print("--- %s seconds ---" % (time.time() - start_time))

# for x,y in con.items():
#     print(x,y)
# print(super_networks.items())
# print(network)

--- 0.0001544952392578125 seconds ---


In [10]:
for x,y in con.items():
    print(x,y)

0 {'downstream': None, 'length': 425, 'upstreams': {16}, 'nid': 0, 'node_order': 0}
1 {'downstream': 4, 'length': 678, 'upstreams': {-999}, 'nid': 0, 'node_order': 3}
3 {'downstream': 0, 'length': 407, 'upstreams': {17}, 'nid': 0, 'node_order': 2}
4 {'downstream': 0, 'length': 798, 'upstreams': {1, 5}, 'nid': 0, 'node_order': 2}
5 {'downstream': 4, 'length': 679, 'upstreams': {-999}, 'nid': 0, 'node_order': 3}
6 {'downstream': 0, 'length': 394, 'upstreams': {-999}, 'nid': 0, 'node_order': 2}
7 {'downstream': 0, 'length': 407, 'upstreams': {18}, 'nid': 0, 'node_order': 2}
8 {'downstream': None, 'length': 420, 'upstreams': {19}, 'nid': 8, 'node_order': 0}
9 {'downstream': 12, 'length': 524, 'upstreams': {10}, 'nid': 8, 'node_order': 3}
10 {'downstream': 9, 'length': 458, 'upstreams': {14}, 'nid': 8, 'node_order': 4}
11 {'downstream': 8, 'length': 548, 'upstreams': {-999}, 'nid': 8, 'node_order': 2}
12 {'downstream': 8, 'length': 543, 'upstreams': {9}, 'nid': 8, 'node_order': 2}
13 {'down